In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


### Scraping Chicago 2022-2023 Crime Data

In [13]:
# import requests
# import pandas as pd
# import json 

# url = "https://data.cityofchicago.org/resource/9hwr-2zxp.json"
# total_rows = 60000
# chunk_size = 1000
# offset = 0
# dataframes = []

# while offset < total_rows:
#     params = {'$limit': chunk_size, '$offset': offset}
#     response = requests.get(url, params=params)

#     if response.status_code == 200:
#         data = response.json()
#         if len(data) == 0:  
#             break
#         df = pd.DataFrame(data)
#         dataframes.append(df)
#         offset += chunk_size
#     else:
#         print("Failed to retrieve data. Status code:", response.status_code)
#         break

# final_df = pd.concat(dataframes, ignore_index=True)

# print("Total rows retrieved:", len(final_df))

Total rows retrieved: 60000


In [28]:
from sodapy import Socrata
from datetime import datetime

api_token = "DQdKNTCLRmXZ2yDZBqPY8HBpM"

client = Socrata("data.cityofchicago.org", api_token)

start_date = "2022-01-01T00:00:00.000"
end_date = "2022-12-31T23:59:59.999"

dataset_identifier = "9hwr-2zxp"

query = f"date between '{start_date}' and '{end_date}'"

results = client.get(dataset_identifier, where=query, limit=500000)

client.close()

import pandas as pd
df_2022 = pd.DataFrame.from_records(results)

print(df_2022.head())


                      date district                    block  \
0  2022-12-31T23:59:00.000      010        037XX W OGDEN AVE   
1  2022-12-31T23:55:00.000      015         055XX W ADAMS ST   
2  2022-12-31T23:50:00.000      024         014XX W LUNT AVE   
3  2022-12-31T23:50:00.000      015  055XX W WASHINGTON BLVD   
4  2022-12-31T23:47:00.000      022        091XX S LAFLIN ST   

                  description             location_description  \
0              NON-AGGRAVATED          MEDICAL / DENTAL OFFICE   
1  RECKLESS FIREARM DISCHARGE  RESIDENCE - YARD (FRONT / BACK)   
2                  AUTOMOBILE                            ALLEY   
3     DOMESTIC BATTERY SIMPLE                        APARTMENT   
4                      SIMPLE                           STREET   

                updated_on community_area  iucr ward case_number  year  \
0  2024-01-06T15:41:12.000             29  0281   24    JH105773  2022   
1  2023-11-15T15:41:31.000             25  1477   29    JG202134  2022

In [31]:
df_2022.shape

(239109, 22)

In [24]:
from sodapy import Socrata
from datetime import datetime

api_token = "DQdKNTCLRmXZ2yDZBqPY8HBpM"

client = Socrata("data.cityofchicago.org", api_token)

start_date = "2023-01-01T00:00:00.000"

end_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000")

dataset_identifier = "xguy-4ndq"

query = f"date between '{start_date}' and '{end_date}'"

results = client.get(dataset_identifier, where=query, limit=1000000)

client.close()

import pandas as pd
df_2023= pd.DataFrame.from_records(results)

print(df_2023.head())


                      date                                           location  \
0  2023-12-30T00:00:00.000  {'latitude': '42.004453476', 'human_address': ...   
1  2023-12-30T00:00:00.000  {'latitude': '41.743692876', 'human_address': ...   
2  2023-12-30T00:00:00.000  {'latitude': '41.994304807', 'human_address': ...   
3  2023-12-30T00:00:00.000  {'latitude': '41.760319717', 'human_address': ...   
4  2023-12-30T00:00:00.000  {'latitude': '41.858946094', 'human_address': ...   

  district                  block y_coordinate      latitude  \
0      016      072XX W PRATT AVE      1944652  42.004453476   
1      006        0000X E 83RD ST      1849995  41.743692876   
2      024  019XX W GRANVILLE AVE      1941197  41.994304807   
3      003   074XX S KINGSTON AVE      1856192  41.760319717   
4      012     017XX S HALSTED ST      1891938  41.858946094   

            description location_description               updated_on  \
0               FORGERY            RESIDENCE  2024-01-0

In [32]:
df_2023.shape

(258304, 22)

In [16]:
df.dtypes

date                    datetime64[ns]
location                        object
district                        object
block                           object
y_coordinate                    object
latitude                        object
description                     object
location_description            object
updated_on                      object
community_area                  object
iucr                            object
x_coordinate                    object
ward                            object
case_number                     object
year                            object
domestic                          bool
fbi_code                        object
longitude                       object
beat                            object
primary_type                    object
arrest                            bool
id                              object
dtype: object

In [29]:
df_2022['date'] = pd.to_datetime(df_2022['date'])

min_date = df_2022['date'].min()
max_date = df_2022['date'].max()

print("Minimum Date:", min_date)
print("Maximum Date:", max_date)

###############################

df_2023['date'] = pd.to_datetime(df_2023['date'])
min_date = df_2023['date'].min()
max_date = df_2023['date'].max()

print("Minimum Date:", min_date)
print("Maximum Date:", max_date)

Minimum Date: 2022-01-01 00:00:00
Maximum Date: 2022-12-31 23:59:00


In [33]:
crime_df = pd.concat([df_2022, df_2023])

crime_df.reset_index(drop=True, inplace=True)


In [34]:
crime_df['date'] = pd.to_datetime(crime_df['date'])

In [20]:
crime_df = crime_df.drop('location', axis=1)


### Scraping Property Data

In [28]:
from homeharvest import scrape_property
from datetime import datetime, timedelta

properties_list = []
start_date = datetime.now() - timedelta(days=500)
end_date = datetime.now()
date_range = (end_date - start_date).days
segments = 90 
days_per_request = date_range // segments

for i in range(segments):
    date_from = start_date + timedelta(days=i * days_per_request)
    date_to = date_from + timedelta(days=days_per_request)

    partial_properties = scrape_property(
        location="Chicago, IL",
        listing_type="sold",
        date_from=date_from.strftime("%Y-%m-%d"),
        date_to=date_to.strftime("%Y-%m-%d")
    )

    print(f"Segment {i+1}, Dates: {date_from.strftime('%Y-%m-%d')} to {date_to.strftime('%Y-%m-%d')}, Number of properties: {len(partial_properties)}")
    properties_list.append(partial_properties)

all_properties = pd.concat(properties_list, ignore_index=True)
print(f"Total Number of properties: {len(all_properties)}")

filename = "HomeHarvest_Chicago.csv"
all_properties.to_csv(filename, index=False)


Segment 1, Dates: 2022-08-24 to 2022-08-29, Number of properties: 706
Segment 2, Dates: 2022-08-29 to 2022-09-03, Number of properties: 995
Segment 3, Dates: 2022-09-03 to 2022-09-08, Number of properties: 510
Segment 4, Dates: 2022-09-08 to 2022-09-13, Number of properties: 614
Segment 5, Dates: 2022-09-13 to 2022-09-18, Number of properties: 658
Segment 6, Dates: 2022-09-18 to 2022-09-23, Number of properties: 501
Segment 7, Dates: 2022-09-23 to 2022-09-28, Number of properties: 406
Segment 8, Dates: 2022-09-28 to 2022-10-03, Number of properties: 653
Segment 9, Dates: 2022-10-03 to 2022-10-08, Number of properties: 690
Segment 10, Dates: 2022-10-08 to 2022-10-13, Number of properties: 441
Segment 11, Dates: 2022-10-13 to 2022-10-18, Number of properties: 584
Segment 12, Dates: 2022-10-18 to 2022-10-23, Number of properties: 499
Segment 13, Dates: 2022-10-23 to 2022-10-28, Number of properties: 635
Segment 14, Dates: 2022-10-28 to 2022-11-02, Number of properties: 692
Segment 15, Dat

In [37]:
all_properties.dtypes

property_url      object
mls               object
mls_id            object
status            object
style             object
street            object
unit              object
city              object
state             object
zip_code          object
beds              object
full_baths        object
half_baths        object
sqft              object
year_built        object
days_on_mls       object
list_price        object
list_date         object
sold_price         int64
last_sold_date    object
lot_sqft          object
price_per_sqft    object
latitude          object
longitude         object
stories           object
hoa_fee           object
parking_garage    object
primary_photo     object
alt_photos        object
dtype: object

In [41]:
property_df = all_properties

In [44]:
columns_to_remove = ['primary_photo', 'alt_photos']

property_df = property_df.drop(columns=columns_to_remove)


In [23]:
import sqlite3

In [24]:
conn = sqlite3.connect('chicago-crime-property.db')
cursor = conn.cursor()


In [26]:
final_df['district'] = final_df['district'].astype(int)
final_df['y_coordinate'] = final_df['y_coordinate'].astype(float)
final_df['latitude'] = final_df['latitude'].astype(float)
final_df['x_coordinate'] = final_df['x_coordinate'].astype(float)
final_df['ward'] = final_df['ward'].astype(int)
final_df['year'] = final_df['year'].astype(int)
final_df['domestic'] = final_df['domestic'].astype(int)
final_df['longitude'] = final_df['longitude'].astype(float)
final_df['beat'] = final_df['beat'].astype(int)
final_df['arrest'] = final_df['arrest'].astype(int)

final_df['date'] = final_df['date'].astype(str)
final_df['updated_on'] = final_df['updated_on'].astype(str)

final_df.to_sql('Crime', conn, if_exists='replace', index=False)

conn.commit()


In [36]:
all_properties['sold_price'] = all_properties['sold_price'].fillna(-1).astype(float)
all_properties['price_per_sqft'] = all_properties['price_per_sqft'].fillna(-1).astype(float)
all_properties['latitude'] = all_properties['latitude'].fillna(-1).astype(float)
all_properties['longitude'] = all_properties['longitude'].fillna(-1).astype(float)
all_properties['stories'] = all_properties['stories'].fillna(-1).astype(int)
all_properties['hoa_fee'] = all_properties['hoa_fee'].fillna(-1).astype(float)
all_properties['mls_id'] = all_properties['mls_id'].fillna(-1).astype(int)
all_properties['beds'] = all_properties['beds'].fillna(-1).astype(int)
all_properties['full_baths'] = all_properties['full_baths'].fillna(-1).astype(int)
all_properties['half_baths'] = all_properties['half_baths'].fillna(-1).astype(int)
all_properties['sqft'] = all_properties['sqft'].fillna(-1).astype(int)
all_properties['year_built'] = all_properties['year_built'].fillna(-1).astype(int)
all_properties['days_on_mls'] = all_properties['days_on_mls'].fillna(-1).astype(int)
all_properties['list_price'] = all_properties['list_price'].fillna(-1).astype(int)


all_properties.to_sql('Property', conn, if_exists='replace', index=False)

conn.commit()


In [37]:
all_properties.to_sql('Property', conn, if_exists='replace', index=False)

final_df.to_sql('Crime', conn, if_exists='replace', index=False)

conn.commit()



In [38]:
db_path = 'chicago-crime-property.db'  

In [ ]:
conn = sqlite3.connect('chicago-cirme-property.db')

cursor = conn.cursor()

In [45]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Convert to GeoDataFrame
crime_df['geometry'] = crime_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
property_df['geometry'] = property_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
crime_gdf = gpd.GeoDataFrame(crime_df, geometry='geometry')
property_gdf = gpd.GeoDataFrame(property_df, geometry='geometry')

# Set the coordinate reference system to WGS84 (latitude/longitude)
crime_gdf.set_crs(epsg=4326, inplace=True)
property_gdf.set_crs(epsg=4326, inplace=True)


,property_url,mls,mls_id,status,style,street,unit,city,state,zip_code,beds,full_baths,half_baths,sqft,year_built,days_on_mls,list_price,list_date,sold_price,last_sold_date,lot_sqft,price_per_sqft,latitude,longitude,stories,hoa_fee,parking_garage,geometry
0,https://www.realtor.com/realestateandhomes-detail/7797244224,CHIL,11456282,SOLD,SINGLE_FAMILY,6127 S Tripp Ave,None,Chicago,IL,60629,3,1,-1,912,1945,53,239000,2022-07-07,214500.0,2022-08-29,None,235.0,41.781364,-87.728592,1,-1.0,2,POINT (-87.72859 41.78136)
1,https://www.realtor.com/realestateandhomes-detail/7133125936,CHIL,11429768,SOLD,SINGLE_FAMILY,6238 S Mayfield Ave,None,Chicago,IL,60638,4,2,1,1712,1926,81,329000,2022-06-09,335000.0,2022-08-29,3781,196.0,41.778587,-87.769427,2,-1.0,2,POINT (-87.76943 41.77859)
2,https://www.realtor.com/realestateandhomes-detail/8298921491,CHIL,11358564,SOLD,CONDOS,400 E Randolph St,Apt 1322,Chicago,IL,60601,0,1,-1,650,1964,154,220000,2022-03-28,210000.0,2022-08-29,None,323.0,41.884987,-87.616671,40,430.0,1,POINT (-87.61667 41.88499)
3,https://www.realtor.com/realestateandhomes-detail/7695426483,CHIL,11468987,SOLD,CONDOS,6121 N Winthrop Ave,Apt 1S,Chicago,IL,60660,3,2,1,-1,2008,41,635000,2022-07-19,625000.0,2022-08-29,None,-1.0,41.993031,-87.657910,4,343.0,1,POINT (-87.65791 41.99303)
4,https://www.realtor.com/realestateandhomes-detail/7002373377,CHIL,11435377,SOLD,CONDOS,2838 W Addison St,Unit 2S,Chicago,IL,60618,1,1,-1,750,1930,75,119900,2022-06-15,102750.0,2022-08-29,None,137.0,41.946948,-87.699666,3,185.0,None,POINT (-87.69967 41.94695)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46009,https://www.realtor.com/realestateandhomes-detail/8348109386,CHIL,11893283,SOLD,SINGLE_FAMILY,6010 S Parkside Ave,None,Chicago,IL,60638,3,2,-1,1250,1966,49,279900,2023-09-25,290000.0,2023-11-13,3750,232.0,41.783063,-87.763495,-1,-1.0,2,POINT (-87.76350 41.78306)
46010,https://www.realtor.com/realestateandhomes-detail/8682788852,CHIL,11862304,SOLD,MULTI_FAMILY,1738 W Roscoe St,None,Chicago,IL,60657,7,3,-1,2185,1875,52,755000,2023-09-22,740000.0,2023-11-13,None,339.0,41.943518,-87.672792,-1,0.0,1,POINT (-87.67279 41.94352)
46011,https://www.realtor.com/realestateandhomes-detail/8733918613,CHIL,11851492,SOLD,SINGLE_FAMILY,8240 S Yates Ave,None,Chicago,IL,60617,3,1,2,1520,-1,101,267000,2023-08-04,267000.0,2023-11-13,4373,176.0,41.745081,-87.566479,2,-1.0,2,POINT (-87.56648 41.74508)
46012,https://www.realtor.com/realestateandhomes-detail/8840769332,CHIL,11887205,SOLD,CONDOS,5224 S Michigan Ave,Unit 2,Chicago,IL,60615,3,2,-1,1300,2007,58,190000,2023-09-16,195000.0,2023-11-13,None,150.0,41.799422,-87.623063,3,225.0,None,POINT (-87.62306 41.79942)


In [46]:
# Convert the radius to degrees (approximately)
radius_in_degrees = 800 / 111320  # 111320 meters is roughly equivalent to a degree

# Buffer the properties by the radius
property_gdf['buffered'] = property_gdf.geometry.buffer(radius_in_degrees)

# Perform the spatial join
joined_gdf = gpd.sjoin(property_gdf.set_geometry('buffered'), crime_gdf, how='left', op='intersects')

# Drop the buffer geometry column if not needed
joined_gdf.drop(columns=['buffered'], inplace=True)


/var/folders/pp/77rpffxd5pl6p9fys4m6t7y00000gn/T/ipykernel_17248/3484706904.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  property_gdf['buffered'] = property_gdf.geometry.buffer(radius_in_degrees)
/Users/rashidbaset/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [48]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Convert to GeoDataFrame
crime_df['geometry'] = crime_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
property_df['geometry'] = property_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
crime_gdf = gpd.GeoDataFrame(crime_df, geometry='geometry', crs=4326)
property_gdf = gpd.GeoDataFrame(property_df, geometry='geometry', crs=4326)

# Convert to a local projected CRS (e.g., NAD83 / Illinois East for Chicago)
crime_gdf = crime_gdf.to_crs(epsg=26971)
property_gdf = property_gdf.to_crs(epsg=26971)

# Buffer in meters
property_gdf['buffered'] = property_gdf.geometry.buffer(800)  # 800 meters

# Perform the spatial join
joined_gdf = gpd.sjoin(property_gdf.set_geometry('buffered'), crime_gdf, how='left', predicate='intersects')

# Drop the buffer geometry column if not needed
joined_gdf.drop(columns=['buffered'], inplace=True)


In [49]:
joined_gdf.head(5)

,property_url,mls,mls_id,status,style,street,unit,city,state,zip_code,beds,full_baths,half_baths,sqft,year_built,days_on_mls,list_price,list_date,sold_price,last_sold_date,lot_sqft,price_per_sqft,latitude_left,longitude_left,stories,hoa_fee,parking_garage,geometry,index_right,date,district,block,y_coordinate,latitude_right,description,location_description,updated_on,community_area,iucr,x_coordinate,ward,case_number,year,domestic,fbi_code,longitude_right,beat,primary_type,arrest,id,human_address
0,https://www.realtor.com/realestateandhomes-detail/7797244224,CHIL,11456282,SOLD,SINGLE_FAMILY,6127 S Tripp Ave,None,Chicago,IL,60629,3,1,-1,912,1945,53,239000,2022-07-07,214500.0,2022-08-29,None,235.0,41.781364,-87.728592,1,-1.0,2,POINT (350273.273 567995.839),15161.0,2022-12-07T15:00:00.000,8.0,065XX S KEELER AVE,1860901.0,41.774227,FROM BUILDING,RESIDENCE,2023-01-03T15:40:27.000,65,0890,1149497.0,13.0,JF501919,2022.0,0.0,06,-87.727527,833.0,THEFT,0.0,12915488,"{""address"": """", ""city"": """", ""state"": """", ""zip"": """"}"
0,https://www.realtor.com/realestateandhomes-detail/7797244224,CHIL,11456282,SOLD,SINGLE_FAMILY,6127 S Tripp Ave,None,Chicago,IL,60629,3,1,-1,912,1945,53,239000,2022-07-07,214500.0,2022-08-29,None,235.0,41.781364,-87.728592,1,-1.0,2,POINT (350273.273 567995.839),863.0,2022-12-30T20:00:00.000,8.0,065XX S KEELER AVE,1860962.0,41.774394,ATTEMPT - AUTOMOBILE,STREET,2023-11-15T15:41:31.000,65,0920,1149495.0,13.0,JG101099,2022.0,0.0,07,-87.727533,833.0,MOTOR VEHICLE THEFT,0.0,12939336,"{""address"": """", ""city"": """", ""state"": """", ""zip"": """"}"
0,https://www.realtor.com/realestateandhomes-detail/7797244224,CHIL,11456282,SOLD,SINGLE_FAMILY,6127 S Tripp Ave,None,Chicago,IL,60629,3,1,-1,912,1945,53,239000,2022-07-07,214500.0,2022-08-29,None,235.0,41.781364,-87.728592,1,-1.0,2,POINT (350273.273 567995.839),52721.0,2022-10-16T09:00:00.000,8.0,064XX S KEDVALE AVE,1861509.0,41.775889,FORGERY,RESIDENCE,2023-01-03T15:40:27.000,65,1120,1149814.0,13.0,JF468036,2022.0,0.0,10,-87.726349,813.0,DECEPTIVE PRACTICE,0.0,12887253,"{""address"": """", ""city"": """", ""state"": """", ""zip"": """"}"
0,https://www.realtor.com/realestateandhomes-detail/7797244224,CHIL,11456282,SOLD,SINGLE_FAMILY,6127 S Tripp Ave,None,Chicago,IL,60629,3,1,-1,912,1945,53,239000,2022-07-07,214500.0,2022-08-29,None,235.0,41.781364,-87.728592,1,-1.0,2,POINT (350273.273 567995.839),21186.0,2022-11-29T05:50:00.000,8.0,065XX S KILDARE AVE,1860986.0,41.774473,AGGRAVATED - HANDGUN,STREET,2023-01-03T15:40:27.000,65,051A,1148829.0,13.0,JF493204,2022.0,0.0,04A,-87.729974,833.0,ASSAULT,0.0,12908268,"{""address"": """", ""city"": """", ""state"": """", ""zip"": """"}"
0,https://www.realtor.com/realestateandhomes-detail/7797244224,CHIL,11456282,SOLD,SINGLE_FAMILY,6127 S Tripp Ave,None,Chicago,IL,60629,3,1,-1,912,1945,53,239000,2022-07-07,214500.0,2022-08-29,None,235.0,41.781364,-87.728592,1,-1.0,2,POINT (350273.273 567995.839),33778.0,2022-11-10T11:45:00.000,8.0,064XX S KOMENSKY AVE,1861673.0,41.776326,AUTOMOBILE,STREET,2023-01-03T15:40:27.000,65,0910,1150476.0,13.0,JF469516,2022.0,0.0,07,-87.723918,813.0,MOTOR VEHICLE THEFT,0.0,12888500,"{""address"": """", ""city"": """", ""state"": """", ""zip"": """"}"


In [50]:
joined_gdf.shape

(15052638, 51)

In [51]:
print("Number of properties:", len(property_gdf))
print("Number of crimes:", len(crime_gdf))
print("Number of joined records:", len(joined_gdf))


Number of properties: 46014
Number of crimes: 60000
Number of joined records: 15052638


In [52]:
print(joined_gdf.isnull().sum())


property_url                   0
mls                      3235072
mls_id                         0
status                         0
style                       1657
street                         0
unit                     5040775
city                           0
state                          0
zip_code                       2
beds                           0
full_baths                     0
half_baths                     0
sqft                           0
year_built                     0
days_on_mls                    0
list_price                     0
list_date                3235072
sold_price                     0
last_sold_date                 0
lot_sqft                10236750
price_per_sqft                 0
latitude_left                  0
longitude_left                 0
stories                        0
hoa_fee                        0
parking_garage           6502487
geometry                       0
index_right                  468
date                         468
district  